In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/  # 移動文件到正確目錄
!chmod 600 ~/.kaggle/kaggle.json  # 設置權限

!pip install kaggle

!kaggle competitions download -c binarycl

!unzip binarycl.zip

Archive:  binarycl.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
df = pd.read_csv("/content/train.csv")

df.drop(columns=["id", "CustomerId", "Surname"], axis = 1, inplace = True)

from sklearn import preprocessing

label_encoder  = preprocessing.LabelEncoder()
df['Geography'] = label_encoder.fit_transform(df['Geography'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])

df.dropna(how = "all", inplace = True)

from sklearn.preprocessing import StandardScaler as SS
scaler = SS()

df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography', 'Gender']] = scaler.fit_transform(df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography', 'Gender']])

X = df.iloc[:, 0:9].values
y = df.iloc[:, 10].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import randint, uniform


# 创建随机森林分类器
rf = RandomForestClassifier(random_state=42)

# 设置超参数分布
param_dist = {
    'n_estimators': randint(100, 500),         # 树的数量
    'max_depth': [None, 10, 20, 30, 40, 50],   # 树的最大深度
    'min_samples_split': randint(2, 20),       # 内部节点再划分所需的最小样本数
    'min_samples_leaf': randint(1, 20),        # 叶子节点所需的最小样本数
    'max_features': ['auto', 'sqrt', 'log2'],  # 每棵树考虑的特征数量
    'bootstrap': [True, False]                  # 是否使用自助法
}

# 创建 RandomizedSearchCV 对象
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=100, cv=3, n_jobs=-1, verbose=2, random_state=42)

# 训练模型
random_search.fit(X_train, y_train)

# 获取最佳参数
best_params = random_search.best_params_
print(f'Best Parameters: {best_params}')

# 使用最佳参数重新训练随机森林模型
best_rf = RandomForestClassifier(**best_params, random_state=42)
best_rf.fit(X_train, y_train)

# 进行预测
y_pred = best_rf.predict(X_test)

# 评估模型
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
99 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
99 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

Best Parameters: {'bootstrap': True, 'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 18, 'n_estimators': 483}
Accuracy: 0.89
Classification Report:
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93      2411
         1.0       0.78      0.61      0.69       589

    accuracy                           0.89      3000
   macro avg       0.85      0.78      0.81      3000
weighted avg       0.88      0.89      0.89      3000

Confusion Matrix:
[[2311  100]
 [ 229  360]]


In [ ]:
df = pd.read_csv("/content/test.csv")

df.drop(columns=["id", "CustomerId", "Surname"], axis = 1, inplace = True)

from sklearn import preprocessing

label_encoder  = preprocessing.LabelEncoder()
df['Geography'] = label_encoder.fit_transform(df['Geography'])
df['Gender'] = label_encoder.fit_transform(df['Gender'])

df.dropna(how = "all", inplace = True)

from sklearn.preprocessing import StandardScaler as SS
scaler = SS()

df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography', 'Gender']] = scaler.fit_transform(df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography', 'Gender']])

X = df.iloc[:, 0:9].values

In [ ]:
y = best_rf.predict(X)

In [ ]:
y

array([0., 1., 0., ..., 1., 0., 0.])

In [ ]:
df = pd.read_csv("/content/test.csv")
df_left_only = df.iloc[:, [0]]
# 将 ndarray 转换为 DataFrame
df_array = pd.DataFrame(y, columns=['Exited'])  # 替换为适当的列名

result = pd.concat([df_left_only, df_array], axis=1)

result

,id,Exited
0,15000,0.0
1,15001,1.0
2,15002,0.0
3,15003,0.0
4,15004,0.0
...,...,...
9995,24995,0.0
9996,24996,0.0
9997,24997,1.0
9998,24998,0.0


In [ ]:
result.to_csv('/content/result_inRandomForest.csv', index=False)